In [1]:
import contextlib
import datetime
import io
import os
import json
import logging
import cv2
import random
import numpy as np
import copy,torch,torchvision
import PIL
from PIL import Image
import math
from itertools import repeat

from fvcore.common.file_io import PathManager
from fvcore.common.timer import Timer

from detectron2.structures import Boxes, BoxMode, PolygonMasks
from detectron2.config import *
from detectron2.modeling import build_model
from detectron2 import model_zoo
from detectron2.data import transforms as T
from detectron2.data import detection_utils as utils
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader, build_detection_train_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.evaluation import RotatedCOCOEvaluator,DatasetEvaluators, inference_on_dataset, coco_evaluation,DatasetEvaluator
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer

import matplotlib.patches as patches
import matplotlib.pyplot as plt
import matplotlib.patches as patches

ModuleNotFoundError: No module named 'cv2'

In [ ]:
class_name_list = ['Right_Index', 'Right_Middle', 'Right_Ring', 'Right_Little', 'Left_Index', 'Left_Middle', 'Left_Ring', 'Left_Little', 'Right_Thumb', 'Left_Thumb']
MetadataCatalog.get('Test').set(thing_classes=class_name_list)

In [ ]:
cfg = get_cfg()

# cfg.OUTPUT_DIR = os.path.join(dataset_path, 'output')
cfg.OUTPUT_DIR = os.path.join('trained_model')

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo

# #used faster_rcnn_R_50_FPN_3x as backbone arch
# cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml") # Let training initialize from model zoo

# #used faster_rcnn_R_101_FPN_3x as backbone arch
# cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml") # Let training initialize from model zoo


cfg.DATASETS.TRAIN = (["Train"])
cfg.DATASETS.TEST = (["Val"])

cfg.MODEL.MASK_ON=False
cfg.MODEL.PROPOSAL_GENERATOR.NAME = "RRPN"
cfg.MODEL.RPN.HEAD_NAME = "StandardRPNHead"
cfg.MODEL.RPN.BBOX_REG_WEIGHTS = (10,10,5,5,1)
cfg.MODEL.ANCHOR_GENERATOR.NAME = "RotatedAnchorGenerator"
cfg.MODEL.ANCHOR_GENERATOR.ANGLES = [[-90,-60,-30,0,30,60,90]]
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8 
cfg.MODEL.ROI_HEADS.NAME = "RROIHeads"
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   #this is far lower than usual.  
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(class_name_list)
cfg.MODEL.ROI_BOX_HEAD.POOLER_TYPE = "ROIAlignRotated"
cfg.MODEL.ROI_BOX_HEAD.BBOX_REG_WEIGHTS = (10,10,5,5,1)
cfg.MODEL.ROI_BOX_HEAD.NUM_CONV=4
cfg.MODEL.ROI_MASK_HEAD.NUM_CONV=8
#cfg.SOLVER.IMS_PER_BATCH = 6 #can be up to  24 for a p100 (6 default)
cfg.SOLVER.IMS_PER_BATCH = 1 #can be up to  24 for a p100 (6 default)
cfg.SOLVER.CHECKPOINT_PERIOD=1500
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.GAMMA=0.5
cfg.SOLVER.STEPS=[1000,2000,4000, 8000, 12000]
cfg.SOLVER.MAX_ITER=14000


cfg.DATALOADER.NUM_WORKERS = 4
cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = True 
cfg.DATALOADER.SAMPLER_TRAIN= "RepeatFactorTrainingSampler"
cfg.DATALOADER.REPEAT_THRESHOLD=0.01
# os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)#lets just check our output dir exists
cfg.MODEL.BACKBONE.FREEZE_AT=6

In [ ]:
# As of 0.3 the XYWHA_ABS box is not supported in the visualizer, this is fixed in master branch atm (19/11/20)
class myVisualizer(Visualizer):
  
    def draw_dataset_dict(self, dic):
        annos = dic.get("annotations", None)
        print("annos before", annos)
        if annos:
            if "segmentation" in annos[0]:
                masks = [x["segmentation"] for x in annos]
            else:
                masks = None
            if "keypoints" in annos[0]:
                keypts = [x["keypoints"] for x in annos]
                keypts = np.array(keypts).reshape(len(annos), -1, 3)
            else:
                keypts = None
            print("annos", annos)
            boxes = [BoxMode.convert(x["bbox"], x["bbox_mode"], BoxMode.XYWHA_ABS) for x in annos]
            print("draw_dataset_dict boxes\n", boxes)
            labels = [x["category_id"] for x in annos]
            print("labels", labels)
            names = self.metadata.get("thing_classes", None)
            print("names", names)
            if names:
                labels = [names[i] for i in labels]
            labels = [
                "{}".format(i) + ("|crowd" if a.get("iscrowd", 0) else "")
                for i, a in zip(labels, annos)
            ]
            self.overlay_instances(labels=labels, boxes=boxes, masks=masks, keypoints=keypts)

        sem_seg = dic.get("sem_seg", None)
        if sem_seg is None and "sem_seg_file_name" in dic:
            sem_seg = cv2.imread(dic["sem_seg_file_name"], cv2.IMREAD_GRAYSCALE)
        if sem_seg is not None:
            self.draw_sem_seg(sem_seg, area_threshold=0, alpha=0.5)
        return self.output

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_0013499.pth")

predictor = DefaultPredictor(cfg)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7 #set threshold value to filter our low scordign bboxes

In [ ]:
imname = 'test_image/Image_00_1_1.bmp'

im = cv2.imread(imname)
outputs = predictor(im)  
# print(outputs)  

v = myVisualizer(im[:, :, ::-1],
              metadata=MetadataCatalog.get("Test"), 
              scale=0.2)

out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.imshow(out.get_image()[:, :, ::-1])
plt.show()